In [27]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.utils as vutils
import random
import os
import shutil
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import numpy as np
import os
import pdb

In [2]:
class make_dataset(Dataset):

    def __init__(self, X, Y, transform=None, mode='train'):
        self.patches = X
        self.labels = Y
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        
        if self.mode == 'train':
            return self.transform(self.patches[idx]),self.labels[idx]
        else:
            return self.transform(self.patches[idx])

In [3]:
# Set Defaults
num_epochs = 10 
batch_size = 32
num_classes = 10 # number of classes for dataset
lr = 0.0002 
b1 = 0.5 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient
n_cpu = 8 # number of cpu threads to use during batch generation
latent_dim = 100 # dimensionality of the latent space
img_size = 256 # size of each image dimension
channels = 1 # number of output image channels
sample_interval = 400 # interval between image sampling

In [4]:
# Set cuda
if torch.cuda.is_available():
    cuda = True 
else:
    cuda = False

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.zero_()

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.use_gpu = cuda
        drop_rate = 0.20
        num_classes = 2
        input_channels = 3
        ndf = 16
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=ndf, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=ndf, out_channels=ndf*2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*2, out_channels=ndf*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*4, out_channels=ndf*8, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        
        # The height and width of downsampled image
        ds_size = img_size // 2**4
        
        # Output layer
        self.linear1 = nn.Sequential(
            nn.Linear(in_features=128*ds_size**2, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        #print("---------------------------------------")
        #print("input size = ", x.size())
        x = self.conv1(x)
        #print("Conv1 output = ", x.size())
        x = self.conv2(x)
        #print("Conv2 output = ", x.size())
        x = self.conv3(x)
        #print("Conv3 output = ", x.size())
        x = self.conv4(x)
        #print("Conv4 output = ", x.size())
        
        # Flatten the features to give as input to the fc linear classification layer
        out = x.view(x.shape[0], -1)
        #print("Out flatten shape = ", out.size())
        out = self.linear1(out)
        #print("Linear output = ", out.size())
        #print("---------------------------------------")
        
        return out

In [7]:
# Initialize loss and model
loss = nn.BCELoss()
model = Model()

if cuda:
    loss.cuda()
    model = nn.DataParallel(model)
    model.cuda()

In [8]:
# Initialize weights
model.apply(weights_init_normal)
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=lr,betas=(b1, b2))

In [9]:
def split_data(data):
    #import pdb;pdb.set_trace()
    n = data['X'].shape[0]
    data_X = data['X']
    data_Y = data['Y']
    train_ratio = 0.7
    valid_ratio = 0.1
    test_ratio = 0.2
    
    train_data = data_X[:int(train_ratio*n)]
    train_label = data_Y[:int(train_ratio*n)]
    valid_data = data_X[int(train_ratio*n):int(train_ratio*n)+int(valid_ratio*n)]
    valid_label = data_Y[int(train_ratio*n):int(train_ratio*n)+int(valid_ratio*n)]
    test_data = data_X[int(train_ratio*n)+int(valid_ratio*n):]
    test_label = data_X[int(train_ratio*n)+int(valid_ratio*n):]
    
    return train_data,train_label, valid_data,valid_label, test_data,test_label

In [10]:
# Load Data
dataset_path = '../dataset/training_data.npz'
data = np.load(dataset_path)
train_data,train_label, valid_data,valid_label,test_data,test_label = split_data(data)
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
    ])

In [11]:
training_data = make_dataset(train_data,train_label, transformations)
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

validation_data = make_dataset(valid_data,valid_label, transformations)
valid_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

test_data = make_dataset(test_data, test_label, transformations)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [28]:
# Training Function
def train(optimizer, criterion, dataloader, model):
    model.train()
    
    epoch_loss = 0
    
    for i, (imgs, labels) in enumerate(dataloader):
        pdb.set_trace()
        
        imgs = Variable(imgs.float())
        labels = Variable(labels.float())

        if cuda:
            imgs = imgs.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    return np.mean(epoch_loss)

In [29]:
# Testing Function
def test(criterion, dataloader, model, mode = None):
    model.eval()
    
    epoch_loss = 0
    total = 0
    correct = 0
    
    for i, (imgs, labels) in enumerate(dataloader):
        imgs = Variable(imgs.float())
        labels = Variable(labels.float())
        
        if cuda:
            imgs = imgs.cuda()
            labels = labels.cuda()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        if mode =='test':
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.long()).sum()
                
    epoch_loss = np.mean(epoch_loss)
    
    if mode == 'test':
        test_accuracy = 100 * correct / total
        return epoch_loss,test_accuracy
    
    return epoch_loss

In [ ]:
for epoch in range(num_epochs):
    train_loss = train(optimizer, loss, train_loader, model)
    valid_loss = test(loss, valid_loader, model)
    test_loss, test_accuracy = test(loss, valid_loader, model,mode='test')
    
    # Display Progress
    print ("[Epoch %d/%d], Train Loss = %f, Validation Loss = %f, Test Loss = %f, Test Accuracy = %f" % (epoch, num_epochs, train_loss, 
                                                                                    valid_loss, test_loss, test_accuracy))
    

> <ipython-input-28-4c719dd0c674>(10)train()
-> imgs = Variable(imgs.float())
(Pdb) n
> <ipython-input-28-4c719dd0c674>(11)train()
-> labels = Variable(labels.float())
(Pdb) n
> <ipython-input-28-4c719dd0c674>(13)train()
-> if cuda:
(Pdb) labels
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
